In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install git+https://github.com/rcabanasdepaz/exPandas.git@main
!pip3 install logging-reset


In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import logreset

import conf
#from experiments import *

In [2]:
conf.prj_path

PosixPath('/root/dev/2021-SIPTA-crema')

In [3]:
# Set the project path as the working directory
if os.getcwd() != conf.prj_path: os.chdir(conf.prj_path)    
rebuild = True

In [ ]:
!mvn clean package 

In [4]:
import subprocess
import datetime
import os

import pandas as pd



def gen_exec(cmd, check_return: bool = False):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line
    popen.stdout.close()
    return_code = popen.wait()
    if return_code and check_return:
        raise subprocess.CalledProcessError(return_code, cmd)


def exec_bash(cmd: str, check_return: bool = False):
    return [s for s in gen_exec(cmd.split(), check_return)]
    
def exec_bash_print(cmd: str, check_return: bool = False):
    for path in gen_exec(cmd.split(), check_return):
        print(path, end="")

exec_bash_print("ls")

MyLogFile.log
MyLogFile12.log
MyLogFile12.log.lck
MyLogFile2.log
README.md
lib
logging.log
logs
networks
nohup.out
notebooks
pom.xml
queries
results
scripts
src
target
tmp


In [5]:
import logging
import logreset
from datetime import datetime


def setup():
    strtime = datetime.now().strftime("%Y%m%d_%H%M%S")
    logreset.reset_logging()
    filename=f'{conf.log_folder}{strtime}.log'
    logging.basicConfig(filename=filename, 
                        format='[%(asctime)s][%(levelname)s][java] %(message)s', 
                        datefmt='%Y-%m-%d_%H:%M:%S',  level=logging.DEBUG)
    
    logging.info(f"Logging info to {filename}")
    
    return filename, strtime

logfile, strtime = setup()


In [6]:
!cat $logfile

[2021-02-18_13:08:51][INFO][java] Logging info to /root/dev/2021-SIPTA-crema/logs/20210218_130851.log


In [25]:
filename

'/root/dev/2021-SIPTA-crema/logs/logging.log'

In [44]:

modelfile = "./networks/vmodel/vmodel-mult_n4_mID2_mD6_mV4_nV4-1.uai"
measure_time = False
method = "cve"
runs=1
warmups=0
target = 0
observed ="1"
timeout = 600


def run_crema(filename:str,  target:int = 0, observed:str ="1", method:str = "cve",
              measure_time:bool = True, runs:int=1, warmups:int=0, timeout:int = 600, **kwargs):
    
    observed=str(observed)
    javafile=f"{conf.java_src}RunCrema.java"
    cmd = "-t" if measure_time else ""
    cmd += f" --method={method} -r {int(runs)} -w {int(warmups)} -x {int(target)}"
    if len(observed)>0: cmd+=f" -y {observed}"
    cmd += f" --timeout={timeout} {modelfile}"
    cmd= f"java -cp {conf.jar_file} {javafile} {cmd}"
    #print(cmd)

    output = exec_bash(cmd)
    
    exec(output[-1])
    return locals()["results"]


run_crema(modelfile)

{'time': 57.0,
 'post': [0.0002807225486488277,
  0.943553381223681,
  0.00013122994269625834,
  0.9936257387232595,
  0.0,
  0.9766375627904647,
  0.0006714725888029766,
  0.9964366523500481],
 'err_msg': '',
 'arg_str': '-t;--method=cve;-r;1;-w;0;-x;0;-y;1;--timeout=600;./networks/vmodel/vmodel-mult_n4_mID2_mD6_mV4_nV4-1.uai'}

In [45]:
def get_model_path(vertex, kind, nodes, max_degree, max_indegree, max_values, num_vert, idx, **kwargs):
    p  = f""
    p += "vmodel/vmodel" if vertex else "hmodel/hmodel"
    p += f"-{kind}_n{nodes}_mID{max_indegree}_mD{max_degree}_mV{max_values}_nV{num_vert}-{idx}.uai"
    return p

def read_query(filename, query_type, field, **kwargs):
    p = filename.replace("vmodel/vmodel", query_type)\
                .replace("vmodel/vmodel", query_type)\
                .replace(".uai",".csv")
    out = pd.read_csv(f"{conf.prj_path}/queries/{p}")[field].values[0]
    if np.isnan(out):
        return ""
    return out




params_dict = dict(
    idx = [1,2,3],
    num_vert = [2,4,6],
    max_degree = [6],
    max_indegree = [2,4,6],
    max_values = [4],
    nodes = [4, 5, 6, 7, 8, 9, 10],
    method=["cve"],
    kind = ["sing"],
    query_type = ["marg", "cond"]
)

func_params = {}
transformations = []


import expandas as expd


func_params = dict(
    filename = lambda t : get_model_path(**{**t, **{"vertex":t["method"].startswith("cve")}}),
    target = lambda t : read_query(**{**t, **dict(field="target")}),
    observed = lambda t : read_query(**{**t, **dict(field="observed")}),
    barren = lambda t : t["nodes"] - read_query(**{**t, **dict(field="size")}),
)
transformations = [lambda df: df.drop(columns=["idx"])]
params = expd.generate_input(params_dict, func_params, transformations)[:4]

params

,num_vert,max_degree,max_indegree,max_values,nodes,method,kind,query_type,filename,target,observed,barren
0,2,6,2,4,4,cve,sing,marg,vmodel/vmodel-sing_n4_mID2_mD6_mV4_nV2-1.uai,3,,0
1,2,6,2,4,4,cve,sing,cond,vmodel/vmodel-sing_n4_mID2_mD6_mV4_nV2-1.uai,0,3,0
2,2,6,2,4,5,cve,sing,marg,vmodel/vmodel-sing_n5_mID2_mD6_mV4_nV2-1.uai,4,,0
3,2,6,2,4,5,cve,sing,cond,vmodel/vmodel-sing_n5_mID2_mD6_mV4_nV2-1.uai,0,4,0


In [61]:
# single_experiemnt
t = params.iloc[0]
import expandas as expd




def run(start=0, end=None, **params_dict):

    logfile, strtime = setup()

    
    func_params = dict(
        filename = lambda t : get_model_path(**{**t, **{"vertex":t["method"].startswith("cve")}}),
        target = lambda t : read_query(**{**t, **dict(field="target")}),
        observed = lambda t : read_query(**{**t, **dict(field="observed")}),
        barren = lambda t : t["nodes"] - read_query(**{**t, **dict(field="size")}),
    )
    transformations = [lambda df: df.drop(columns=["idx"])]
    params = expd.generate_input(params_dict, func_params, transformations)
    
    end = end or len(params)
    params = params[start:end]

    count = 0

    def single_experiment(**t):
        global count, logfile

        logging.info((f"Starting experiment {count}"))
        count += 1
        args = t.copy()
        res = run_crema(**args, logfile=logfile)
        output = args
        output["result"] = " ".join([f"{p}" for p in res["post"]]) if len(res["err_msg"])==0 else ""
        return output


    df = pd.DataFrame(expd.Experiment(functions=[single_experiment]).run(params))
    
    return df



params_dict = dict(
    idx = [1,2,3],
    num_vert = [2,4,6],
    max_degree = [6],
    max_indegree = [2,4,6],
    max_values = [4],
    nodes = [4, 5, 6, 7, 8, 9, 10],
    method=["cve"],
    kind = ["sing"],
    query_type = ["marg", "cond"]
)

run(start=0, end=3, **params_dict)

# process output

,num_vert,max_degree,max_indegree,max_values,nodes,method,kind,query_type,filename,target,observed,barren,result
0,2,6,2,4,4,cve,sing,marg,vmodel/vmodel-sing_n4_mID2_mD6_mV4_nV2-1.uai,3,,0,0.048 0.496 0.028 0.154 0.433 0.905
1,2,6,2,4,4,cve,sing,cond,vmodel/vmodel-sing_n4_mID2_mD6_mV4_nV2-1.uai,0,3,0,0.002 0.09500000000000001 0.011 0.834 0.0 0.29...
2,2,6,2,4,5,cve,sing,marg,vmodel/vmodel-sing_n5_mID2_mD6_mV4_nV2-1.uai,4,,0,


In [36]:
args.to_dict()

{'num_vert': 2,
 'max_degree': 6,
 'max_indegree': 2,
 'max_values': 4,
 'nodes': 4,
 'method': 'cve',
 'kind': 'sing',
 'query_type': 'marg',
 'filename': 'vmodel/vmodel-sing_n4_mID2_mD6_mV4_nV2-1.uai',
 'target': 3,
 'observed': '',
 'barren': 0,
 'result': '0.048 0.496 0.028 0.154 0.433 0.905'}

In [195]:
n = 72
m = 5
x = 40


#print("1 - [" + " * ".join([f"(1 - ({x}/{n-i}))" for i in range(0,m)])+"]")
def get_prob(x):
    return 1 - np.prod([1 - x/(n-i) for i in range(0,m)])